In [1]:
!pip install pandas datasets unsloth transformers accelerate bitsandbytes -q


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch-tensorrt 2.5.0a0 requires torch<2.6.0,>=2.5.0.dev, but you have torch 2.7.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch
from tqdm import tqdm


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: If you want to finetune Gemma 2, upgrade flash-attn to version 2.6.3 or higher!
Newer versions support faster and less memory usage kernels for Gemma 2's attention softcapping!
To update flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Load your CSV (make sure the file 'medquad_tamil.csv' is uploaded)
df = pd.read_csv("medquad_tamil.csv").dropna()

# Create a formatted 'text' column for the LLM
df["text"] = "### கேள்வி:\n" + df["question_tamil"] + "\n\n### பதில்:\n" + df["answer_tamil"]

# Split into train and test datasets (keeping original columns)
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))


In [4]:
# Load the model from Hugging Face
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="abhinand/tamil-llama-7b-instruct-v0.2",  # HuggingFace repo ID
    max_seq_length=512,
    dtype=torch.float16,
    load_in_4bit=True,
    device_map="auto",
)

# Fix padding token
tokenizer.pad_token = tokenizer.eos_token
model.eval()


==((====))==  Unsloth 2025.6.3: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.85s/it]
Unsloth: Will load abhinand/tamil-llama-7b-instruct-v0.2 as a legacy tokenizer.


abhinand/tamil-llama-7b-instruct-v0.2 does not have a padding token! Will use pad_token = <unk>.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49994, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4

In [8]:
print(type(test_dataset))
print(test_dataset[0])


<class 'datasets.arrow_dataset.Dataset'>
{'question_tamil': 'இளைஞர் ஹண்டிங்டன் நோய்த்தொற்று என்ன?', 'answer_tamil': 'ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன்', 'text': '### கேள்வி:\nஇளைஞர் ஹண்டிங்டன் நோய்த்தொற்று என்ன?\n\n### பதில்:\nஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன் ஹாண்டிங்டன்'}


In [9]:
from tqdm import tqdm

generated_answers = []
reference_answers = []

# Use Hugging Face's `select` method to get the first 100 samples
subset = test_dataset.select(range(100))

for item in tqdm(subset):
    question = item["question_tamil"]
    reference = item["answer_tamil"]

    prompt = f"""### கேள்வி:
{question}

### பதில்:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    if "### பதில்:" in decoded:
        answer = decoded.split("### பதில்:")[-1].strip()
    else:
        answer = decoded.replace(prompt, "").strip()

    if not answer:
        print(f"⚠️ No answer for question: {question}")
        print("🔍 Full decoded output:\n", decoded)
    else:
        print(f"✅ Answer generated for: {question}")

    generated_answers.append(answer)
    reference_answers.append(reference)


  1%|█▌                                                                                                                                                      | 1/100 [00:03<06:21,  3.85s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ Answer generated for: இளைஞர் ஹண்டிங்டன் நோய்த்தொற்று என்ன?


  2%|███                                                                                                                                                     | 2/100 [00:12<11:20,  6.94s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ Answer generated for: பெர்ரி நோய்க்கான சிகிச்சைகள் என்ன?


  3%|████▌                                                                                                                                                   | 3/100 [00:29<18:33, 11.48s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

✅ Answer generated for: க்ளவுகோமாவுக்கு என்ன சிகிச்சைகள்?


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 99/100 [15:00<00:08,  8.56s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ Answer generated for: ஓஸ்டியோபிரோஸ் 7 - ன் அறிகுறிகள் என்ன?


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [15:17<00:00,  9.17s/it]

✅ Answer generated for: பல மையமான ஓட்டோலிசிஸ், நெடுஞ்சாலை, தொற்று நோய் போன்ற மரபணு மாற்றங்கள் என்னவாகும்?


In [11]:
!pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [12]:
from bert_score import score

P, R, F1 = score(generated_answers, reference_answers, lang="ta")  # ta = Tamil

print(f"Precision: {P.mean().item():.4f}")
print(f"Recall:    {R.mean().item():.4f}")
print(f"F1 Score:  {F1.mean().item():.4f}")


Precision: 0.6018
Recall:    0.6386
F1 Score:  0.6177


In [13]:
import pandas as pd

# Convert tensors to numpy
P = P.numpy()
R = R.numpy()
F1 = F1.numpy()

# Report average
print(f"\n✅ BERTScore Precision: {P.mean():.4f}")
print(f"✅ BERTScore Recall:    {R.mean():.4f}")
print(f"✅ BERTScore F1:        {F1.mean():.4f}")

# Optional: Save individual results
results_df = pd.DataFrame({
    "Generated Answer": generated_answers,
    "Reference Answer": reference_answers,
    "Precision": P,
    "Recall": R,
    "F1 Score": F1
})

results_df.to_csv("not_tamil_bertscore_results.csv", index=False)
print("✅ Saved: not_tamil_bertscore_results.csv")



✅ BERTScore Precision: 0.6018
✅ BERTScore Recall:    0.6386
✅ BERTScore F1:        0.6177
✅ Saved: not_tamil_bertscore_results.csv
